In [1]:
import boto3
import numpy as np
import datetime
import pandas as pd 
import os
import cv2
from datetime import date
import os.path
from functools import cmp_to_key
from dateutil.parser import parse

#remove this
#from IPython.display import Image, display



In [2]:
def license(lines):
    poss_state=['AP','AR','AS','BR','CG','GA','GJ','HR','HP','JH','KA','KL','MP','MH','MN','ML','MZ','NL','OD','PB','RJ','SK',\
                'TN','TS','TR','UP','UK','WB','AN','CH','DD','DL','JK','LA','LD','PY']
    
    for i in range(len(lines)):
        try:
            regstring=''.join(e for e in lines[i].upper() if e.isalnum())            
            lastnum=regstring[-4:]
            if lastnum.isnumeric():
                if(regstring[-9:-7] in poss_state):
                    return regstring[-9:]  
                if(regstring[-8:-6] in poss_state):
                    return regstring[-8:]    
                if(regstring[-10:-8] in poss_state):
                    return regstring[-10:]   
            else:
                pass                    
        except:
            pass

    return None 

In [3]:
def chasis1(words):
    try:
        for wrd in words:
            if(len(wrd))==17:
                lastnum=wrd[-6:]
                if(lastnum.isnumeric()):
                    return wrd
    except:
        pass   
    
    return None

In [4]:
def chasis2(words):
    keys=['ch', 'chasis', 'chassis']
    for i in range(len(words)):
        for key in keys:            
            if key in words[i].lower():
                try:
                    k=1
                    while(k<5):
                        if(len(words[i+k])>13):
                            return words[i+k]
                        k=k+1 
                except:
                    pass

    return None  

In [5]:
def chasis(words):
    for i in range(len(words)):
        words[i]=''.join(e for e in words[i] if e.isalnum()) 
    if chasis1(words) is None:
        return chasis2(words)
    else:
        return chasis1(words)

In [6]:
def name(lines):
    for i in range(len(lines)):
        line=''
        for e in lines[i].lower():
            if not (e.isalnum()):
               line=line+' '
            else:
                line=line+e    
        line.strip()
        #assume name types end with owner or name (ex:name of owner, owner, name, owner's name)
        try:
            if ('name' in line) or ('owner' in line):
                if ('name'==line[-4:] or 'owner'==line[-5:] or 'name'==line[0:4]):
                    return ''.join(e for e in lines[i+1] if e.isalnum() or e==' ') 
                if(line.find('name ')!=-1 or line.find('owner ')!=-1):
                    indn_start=line.find('name ')+5                    
                    indo_start=line.find('owner ')+6
                    ind=max(indn_start,indo_start)                    
                    return line[ind:]  
        except:
                pass
    return None  

In [7]:
def engineno(words, response):
    
    try:
        ch=chasis(words)
        for item in response['Blocks']:
            if item['BlockType'] == 'WORD' and item['Text']==ch:
                bbox=item['Geometry']['BoundingBox']

        bbox_x_left=bbox['Left']
        bbox_y_top=bbox['Top']
        bbox_x_right=bbox['Left']+bbox['Width']

        y_cut=1000         #arbitrary large
        eno=None
        for item in response['Blocks']:
            if item['BlockType'] == 'WORD':
                bbox_item=item['Geometry']['BoundingBox']
                bbox_item_x_right=bbox_item['Left']+(bbox_item['Width']/2)
                if (bbox_item_x_right < bbox_x_right) and  (bbox_x_left < bbox_item_x_right) and (bbox_y_top < bbox_item['Top']):
                    if(bbox_item['Top'] < y_cut):
                        y_cut=bbox_item['Top']
                        eno=item['Text']
        return eno
    
    except:
        pass
    
    return None

In [8]:
def dates(words):
    dates_=[]
    for wrd in words: 
        if any(i in wrd for i in ['/','-']):
            try:
                result = parse(wrd, fuzzy_with_tokens=True, dayfirst=True, default=datetime.datetime(666,1,1))
                if(result[0].year<2060 and result[0].year>1970): 
                    no_day=False
                    #check if the day is provided:
                    if(result[0].day==1):
                        res_temp = parse(wrd, fuzzy_with_tokens=True, dayfirst=True, default=datetime.datetime(666,1,2))
                        if(res_temp[0].day==2):
                            no_day=True
                    if(no_day):
                        dates_.append([ 0, result[0].month, result[0].year])
                    else:
                        dates_.append([result[0].day, result[0].month, result[0].year])
                    #print(wrd)
            except:
                pass
        
    return dates_

In [9]:
def compare(item1, item2):
    if item1[2]>item2[2]:
        return 1
    elif item1[1]>item2[1] and item1[2]==item2[2]:
        return 1
    elif item1[0]>item2[0] and item1[1]==item2[1] and item1[2]==item2[2]:
        return 1
    else:
        return -1

In [10]:
def mfddate(dates_arr):
    try:
        mfg=dates_arr[0]
        if(mfg[0]==0):
            return (str(mfg[1])+'/'+str(mfg[2]))
        else:
            return (str(mfg[0])+'/'+str(mfg[1])+'/'+str(mfg[2]))            
    except:
        return None

In [11]:
def regdate(dates_arr):
    try:
        mfg=dates_arr[1]
        if(mfg[0]==0):
            return (str(mfg[1])+'/'+str(mfg[2]))
        else:
            return (str(mfg[0])+'/'+str(mfg[1])+'/'+str(mfg[2]))            
    except:
        return None

In [12]:
###################################################################

In [ ]:
def newDataframe():
    
    #create a new dataframe table named 'weather.csv'
    df_new = pd.DataFrame(columns = ['Picname', 'LicNo', 'ChNo' , 'ENo', 'Name', 'MfdDate', 'RegDate']) 
    df_new.to_csv('extracted.csv',index=False) 
    
def addtoDataframe(words, lines, pic_name, response):

    dates_sorted=sorted(dates(words), key=cmp_to_key(compare))
    #print(dates_sorted)
    
    data={'Picname':pic_name, 
          'LicNo':license(lines), 
          'ChNo': chasis(words),
          'ENo': engineno(words,response), 
          'Name':name(lines), 
          'MfdDate':mfddate(dates_sorted), 
          'RegDate':regdate(dates_sorted)    
         }
    
    #open 'weather.csv' append data and save
    df_read = pd.read_csv("extracted.csv") 
    df_read=df_read.append(data, ignore_index=True, sort=None)
    df_read.to_csv('extracted.csv',index=False) 
    print(pic_name+' : '+'Done!')
    #print(data)
      
def main():

    newDataframe()   
    
    ###################### FILL ACCESSKEYS AND SECRET ACCESSKEYS FOR AWS TEXTRACT########################
    textract = boto3.client('textract', region_name='us-east-2',
                            aws_access_key_id='xxxxxxxxxxxxxxxxxxxx',
                            aws_secret_access_key='yyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyy')
    #####################################################################################################

    
    imgs=[]
    for x in os.listdir():
        if any(i in x for i in ['.jpg', '.png', '.jpeg']):
            imgs=np.append(imgs,x)      
            
    print('Please wait, contacting API')
    for pic_name in imgs: 
        
        try:
            img=cv2.imread(pic_name,0) 
            red_img = cv2.resize(img,(800,600), interpolation = cv2.INTER_AREA)
            is_success, im_buf_arr = cv2.imencode(".jpg", red_img)
            imgBytes=bytearray(im_buf_arr)

            response = textract.detect_document_text(Document={'Bytes': imgBytes}) 

            #print('################'+pic_name+'###################')
            #i=Image(data=im_buf_arr)
            #display(i)

            words=[]
            for item in response['Blocks']:
                if item['BlockType'] == 'WORD':
                    words.append(item['Text'])

            #print(words)
            #print('################')


            lines=[]
            for item in response['Blocks']:
                if item['BlockType'] == 'LINE':
                    lines.append(item['Text']) 
            #print(lines)                

            addtoDataframe(words,lines,pic_name,response)
            
        except:
            pass

    
#RUN
main()

In [14]:
#PLEASE FILL AWS ACCESS KEYS IN ABOVE CELL CODE FOR TEXTRACT
